In [1]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../../Data/admission_data.csv")
df

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,337,118,4,4.5,4.5,9.65,1,0.92
1,324,107,4,4.0,4.5,8.87,1,0.76
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
...,...,...,...,...,...,...,...,...
495,332,108,5,4.5,4.0,9.02,1,0.87
496,337,117,5,5.0,5.0,9.87,1,0.96
497,330,120,5,4.5,5.0,9.56,1,0.93
498,312,103,4,4.0,5.0,8.43,0,0.73


In [3]:
df.isnull().mean()

GRE Score            0.0
TOEFL Score          0.0
University Rating    0.0
SOP                  0.0
LOR                  0.0
CGPA                 0.0
Research             0.0
Chance of Admit      0.0
dtype: float64

# 1. Data Preparation

In [4]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X_trf = ss.fit_transform(df.iloc[:,:-1])

In [5]:
X_trf

array([[ 1.81923762,  1.77886545,  0.77558214, ...,  1.09894429,
         1.77680627,  0.88640526],
       [ 0.66714832, -0.03160087,  0.77558214, ...,  1.09894429,
         0.48585943,  0.88640526],
       [-0.0418297 , -0.52536441, -0.09979274, ...,  0.01730621,
        -0.95404281,  0.88640526],
       ...,
       [ 1.19888185,  2.10804114,  1.65095702, ...,  1.63976333,
         1.62785086,  0.88640526],
       [-0.39631872, -0.68995225,  0.77558214, ...,  1.63976333,
        -0.24236699, -1.12815215],
       [ 0.93301508,  0.95592621,  0.77558214, ...,  1.09894429,
         0.76721964, -1.12815215]], shape=(500, 7))

In [6]:
y = df.iloc[:,-1]
y

0      0.92
1      0.76
2      0.72
3      0.80
4      0.65
       ... 
495    0.87
496    0.96
497    0.93
498    0.73
499    0.84
Name: Chance of Admit , Length: 500, dtype: float64

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_trf,y,test_size=0.2)

In [8]:
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)
y_test = np.asarray(y_test).astype(np.float32)

In [9]:
X_train

array([[-0.30769646, -0.8545401 , -0.09979273, ..., -1.0643319 ,
         0.17139801, -1.1281521 ],
       [-0.2190742 ,  0.46216267, -0.09979273, ...,  0.55812526,
         0.37000522, -1.1281521 ],
       [ 0.04679255, -0.19618872, -1.8505425 , ...,  0.01730621,
        -1.5333139 ,  0.8864053 ],
       ...,
       [-0.6621855 , -1.3483037 , -0.09979273, ...,  0.55812526,
        -0.02720919, -1.1281521 ],
       [-0.5735632 , -0.36077657, -0.97516763, ...,  0.01730621,
        -0.9374922 , -1.1281521 ],
       [ 1.1102595 ,  1.120514  , -0.97516763, ...,  0.55812526,
        -0.02720919,  0.8864053 ]], shape=(400, 7), dtype=float32)

# 2. Dataset and Dataloader

In [10]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self,features,labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)
        
    def __len__(self):
        return self.features.shape[0]
    
    def __getitem__(self, index):
        return self.features[index], self.labels[index]

In [11]:
train_dataset = CustomDataset(X_train,y_train)
test_dataset = CustomDataset(X_test,y_test)

In [12]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# 3. Model Build

In [13]:
class NeuralNetworkRegressor(nn.Module):
    def __init__(self,X_train):
        super().__init__()
        
        self.network = nn.Sequential(
            nn.Linear(X_train.shape[1], 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.Dropout(0.3),
            
            nn.Linear(32,1)
        )
    def forward(self,x):
        return self.network(x)

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [15]:
epochs = 50
learning_rate = 0.01

mseError = nn.MSELoss()

model = NeuralNetworkRegressor(X_train)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate, weight_decay=1e-5)

In [16]:
for epoch in range(epochs):
    epoch_loss = 0
    
    for batch_features, batch_labels in train_dataloader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        
        y_pred = model(batch_features)
        
        loss = mseError(y_pred, batch_labels)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    print(f"Epoch {epoch+1}: Loss {epoch_loss/len(train_dataloader)}")

D:\anaconda3\envs\jh_env\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
D:\anaconda3\envs\jh_env\Lib\site-packages\torch\nn\modules\loss.py:608: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1: Loss 0.30762629898694843
Epoch 2: Loss 0.11588590133648652
Epoch 3: Loss 0.06601832778407977
Epoch 4: Loss 0.05329828279522749
Epoch 5: Loss 0.04849837233240788
Epoch 6: Loss 0.04150389708005465
Epoch 7: Loss 0.036960181421958484
Epoch 8: Loss 0.03528840524645952
Epoch 9: Loss 0.03380165091500832
Epoch 10: Loss 0.031661166021457084
Epoch 11: Loss 0.03243465750263287
Epoch 12: Loss 0.0296665970236063
Epoch 13: Loss 0.029669697897938583
Epoch 14: Loss 0.028704777359962463
Epoch 15: Loss 0.02721404255582736
Epoch 16: Loss 0.026096522664794557
Epoch 17: Loss 0.02613172995356413
Epoch 18: Loss 0.024709602244771443
Epoch 19: Loss 0.024538518431094978
Epoch 20: Loss 0.023541644645424988
Epoch 21: Loss 0.023944703288949452
Epoch 22: Loss 0.02267227780360442
Epoch 23: Loss 0.02351436162224183
Epoch 24: Loss 0.022617214216062657
Epoch 25: Loss 0.022197502163740303
Epoch 26: Loss 0.023941923219424028
Epoch 27: Loss 0.022764051906191386
Epoch 28: Loss 0.021309056104375765
Epoch 29: Loss 0

In [17]:
from sklearn.metrics import r2_score

all_preds = []
all_labels = []

with torch.no_grad():
    for batch_features, batch_labels in test_dataloader:
        batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
        
        y_pred = model(batch_features)
        all_preds.append(y_pred.cpu())
        all_labels.append(batch_labels.cpu())

all_preds = torch.cat(all_preds).numpy()
all_labels = torch.cat(all_labels).numpy()
r2 = r2_score(all_labels, all_preds)
print("Testing R² Score:", r2)


Testing R² Score: -0.030431628227233887
